# Detecção de textos falando sobre Influenza a partir dos sintomas encontrados

In [1]:
from ontologies.symptom import flu_symptoms, symptoms, all_flu_symptoms
import nltk
from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

## Corpus a ser analisado
- Insira na variável **corpus** o texto que deve ser analisado

In [2]:
corpus = "I have a lot of cough and fever."

In [2]:
corpus = """
Initially, the flu may seem like a common cold with a runny nose, sneezing and sore throat. But colds usually develop slowly, whereas the flu tends to come on suddenly. And although a cold can be a nuisance, you usually feel much worse with the flu.

Common signs and symptoms of the flu include:

Fever over 100.4 F (38 C)
Aching muscles
Chills and sweats
Headache
Dry, persistent cough
Fatigue and weakness
Nasal congestion
Sore throat
"""

In [2]:
corpus = """
Tessa has the swine flu. Luckily not much worse than a cough and on/off fever.  Getting better!  Time feels weird. Been outta work 2 days.
"""

## Etapa de pré-processamento

Nesta etapa será processado o texto de modo a remover stop-words e criar n-grams (2-grams) para ser identificado os sintomas presentes.

In [3]:
def preproc(text):
    tokenizer = RegexpTokenizer(r'\w+')
    stop_words = set(stopwords.words('english'))
    
    text = text.lower()
    tokens = list(filter(lambda w: w not in stop_words, tokenizer.tokenize(text)))
    n_grams_tokens = list(map(lambda s: ' '.join(s), ngrams(tokens, n=2)))
    tokens += n_grams_tokens
    
    return tokens

In [4]:
words = preproc(corpus)
print(words)

['initially', 'flu', 'may', 'seem', 'like', 'common', 'cold', 'runny', 'nose', 'sneezing', 'sore', 'throat', 'colds', 'usually', 'develop', 'slowly', 'whereas', 'flu', 'tends', 'come', 'suddenly', 'although', 'cold', 'nuisance', 'usually', 'feel', 'much', 'worse', 'flu', 'common', 'signs', 'symptoms', 'flu', 'include', 'fever', '100', '4', 'f', '38', 'c', 'aching', 'muscles', 'chills', 'sweats', 'headache', 'dry', 'persistent', 'cough', 'fatigue', 'weakness', 'nasal', 'congestion', 'sore', 'throat', 'initially flu', 'flu may', 'may seem', 'seem like', 'like common', 'common cold', 'cold runny', 'runny nose', 'nose sneezing', 'sneezing sore', 'sore throat', 'throat colds', 'colds usually', 'usually develop', 'develop slowly', 'slowly whereas', 'whereas flu', 'flu tends', 'tends come', 'come suddenly', 'suddenly although', 'although cold', 'cold nuisance', 'nuisance usually', 'usually feel', 'feel much', 'much worse', 'worse flu', 'flu common', 'common signs', 'signs symptoms', 'symptoms

## Encontrando os sintomas com base em ontologias 
- Utilizado as ontologias: SymptomOntology e FluOntology

In [5]:
flu_symp = flu_symptoms(words, n_threads=1)
symp = symptoms(words, n_threads=1)

print(symp)
print(flu_symp)

Exception in thread Thread-6:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/Users/orlando/Projects/predflu/src/ontologies/symptom.py", line 87, in run
    self._return = self._target(*self._args, **self._kwargs)
  File "/Users/orlando/Projects/predflu/src/ontologies/symptom.py", line 98, in <lambda>
    filter_fn = lambda ch: set(filter(fn, ch))
  File "/Users/orlando/Projects/predflu/src/ontologies/symptom.py", line 40, in is_symptom
    res = symp_endpoint.execute_query(query)
  File "/Users/orlando/Projects/predflu/src/ontologies/sparql.py", line 17, in execute_query
    res = self.graph.query(query)
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/rdflib/graph.py", line 1089, in query
    query_object, initBindings, initNs, **kwargs))
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-package

set()
{'fatigue', 'cough', 'runny nose', 'headache', 'nasal congestion', 'fever'}


Exception in thread Thread-17:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/Users/orlando/Projects/predflu/src/ontologies/symptom.py", line 87, in run
    self._return = self._target(*self._args, **self._kwargs)
  File "/Users/orlando/Projects/predflu/src/ontologies/symptom.py", line 98, in <lambda>
    filter_fn = lambda ch: set(filter(fn, ch))
  File "/Users/orlando/Projects/predflu/src/ontologies/symptom.py", line 40, in is_symptom
    res = symp_endpoint.execute_query(query)
  File "/Users/orlando/Projects/predflu/src/ontologies/sparql.py", line 17, in execute_query
    res = self.graph.query(query)
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/rdflib/graph.py", line 1089, in query
    query_object, initBindings, initNs, **kwargs))
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packag

### Aplicação da similaridade de Jaccard entre os sintomas

In [13]:
def jaccard(a, b):
    i = len(a.intersection(b))
    u = len(a.union(b))
    return i / u if u > 0 else 1.

- Similaridade entre todos os sintomas do texto com os sintomas de influenza contidos no texto

In [14]:
jaccard(symp, flu_symp)

1.0

- Similaridade entre os sintomas de influenza contidos no texto e todos presentes na ontologia

In [15]:
jaccard(flu_symp, all_flu_symptoms())

0.25

## Analisando mensagens do twitter coletadas

In [4]:
twitter_texts = []
twitter_fp = '../twitter-data/tweets.csv'
with open(twitter_fp, 'r') as fin:
    lines = map(lambda l: l.split(','), fin.read().split('\n')[1:-1])
    tweets = list(map(lambda l: l[1] if len(l) >= 2 else l[0], lines))

In [5]:
print('number of tweets: {}'.format(len(tweets)))

number of tweets: 4523


- Para cada twitter calcular as duas similaridades apresentadas anteriormente entre os sintomas

In [8]:
tweets_metrics = []
all_flu_symps = all_flu_symptoms()
for i, tweet in enumerate(tweets):
    words = preproc(tweet)
    flu_symp = flu_symptoms(words, n_threads=1)
    flu_symptoms_sim = jaccard(flu_symp, all_flu_symps)
    tweets_metrics.append((tweet, flu_symptoms_sim))

In [9]:
tweets_metrics.sort(key=lambda t: t[1], reverse=True)

In [12]:
for t, m in tweets_metrics[:10]:
    print('Tweet: {} Sim: {}'.format(t, m))

Tweet: Tessa has the swine flu. Luckily not much worse than a cough and on/off fever.  Getting better!  Time feels weird. Been outta work 2 days. Sim: 0.25
Tweet: 's whole family is freaking out thinking that I have the swine flu.  since when is a headache and a fever the swine flu? Sim: 0.25
Tweet: "I hav a headache and a hard cough where u cough (not often) but wen u do Sim: 0.25
Tweet: "@GraceyJones I've been hugging her most of the day!  Motrin's just reducing fever Sim: 0.125
Tweet: H1N1 Flu Update: Better Not Cough! Santas Want Swine Flu Shots: Santas across the nation worry .. http://bit.ly/2RS6Xn Sim: 0.125
Tweet: "headache from hell Sim: 0.125
Tweet: I have to take a shot of cough syrup. This coughing is getting to be a bit much. This 2day flu is no joke. At least I hope its 2day flu. :-) Sim: 0.125
Tweet: Hagan official has the flu. Fever of 101 and he keeps getting sick. Sim: 0.125
Tweet: i'm getting flu and lil bit fever.. GOD gv me a power to survive. Sim: 0.125
Tweet: I a